<a href="https://colab.research.google.com/github/gabrieldimas/periapical-photo-caries-detection-using-deep-learning/blob/main/dataset_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import os

zip_path = "/content/dataset_v5_mix.zip"
extract_to = "/content/dataset"

os.makedirs(extract_to, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"File berhasil diekstrak ke: {extract_to}")

File berhasil diekstrak ke: /content/dataset


In [4]:
# 1. Install ultralytics (jika belum)
!pip install ultralytics

# 2. Import library
import os
import cv2
from PIL import Image
import numpy as np
from ultralytics import YOLO
from glob import glob

# 3. Load model
model = YOLO('/content/best.pt')

# 4. Buat folder hasil cropping jika belum ada
output_dir = '/content/segmented'
os.makedirs(output_dir, exist_ok=True)

# 5. Ambil semua gambar dari /content/dataset
image_paths = glob('/content/dataset/dataset_v5 _mix/*.jpg') + glob('/content/dataset/dataset_v5 _mix/*.png')  # support jpg dan png

# 6. Loop setiap gambar dan lakukan deteksi + crop
for img_path in image_paths:
    img_name = os.path.basename(img_path).split('.')[0]

    # Baca gambar
    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Deteksi objek
    results = model.predict(source=image_rgb, save=False, verbose=False)[0]

    # Ambil koordinat bounding box
    boxes = results.boxes.xyxy.cpu().numpy()

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box.astype(int)

        # Crop gigi
        cropped = image[y1:y2, x1:x2]

        # Simpan hasil crop
        crop_filename = f"{img_name}_tooth{i+1}.jpg"
        cv2.imwrite(os.path.join(output_dir, crop_filename), cropped)

print("✅ Semua gigi berhasil dicrop dan disimpan di:", output_dir)

✅ Semua gigi berhasil dicrop dan disimpan di: /content/segmented


In [5]:
# Hitung jumlah file gambar dalam folder /content/segmented
segmented_dir = '/content/segmented'
image_extensions = ['.jpg', '.jpeg', '.png']

# Filter hanya file gambar
image_files = [f for f in os.listdir(segmented_dir) if os.path.splitext(f)[1].lower() in image_extensions]
print(f"Jumlah gambar hasil crop di folder segmented: {len(image_files)}")


Jumlah gambar hasil crop di folder segmented: 4636


In [6]:
import shutil

folder_path = "/content/segmented"
zip_path = "/content/segmented_teeth.zip"

shutil.make_archive(zip_path.replace(".zip", ""), 'zip', folder_path)

from google.colab import files
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>